In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# Import Library

In [2]:
# sub package download
!pip install -q mxnet
!pip install -q gluonnlp pandas tqdm
!pip install -q sentencepiece
!pip install -q torch
!pip install -q transformers==3

     |████████████████████████████████| 55.0MB 57kB/s 
     |████████████████████████████████| 348kB 12.1MB/s 
     |████████████████████████████████| 1.1MB 20.7MB/s 
     |████████████████████████████████| 757kB 13.3MB/s 
     |████████████████████████████████| 3.0MB 57.6MB/s 
     |████████████████████████████████| 890kB 56.4MB/s 


In [3]:
# model structure download
!pip install -q git+https://git@github.com/SKTBrain/KoBERT.git@master

In [4]:
# os path
import os

# data analysis
import tensorflow as tf
import pandas as pd

# nlp
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
BASE_DIR = "/gdrive/My Drive/datascience/goverment_hackerton/emotion_analysis" # 로컬 경로로 바꿔주어야 로컬에서 작동됨

CODE_DIR = os.path.join(BASE_DIR, "code")
DATA_DIR = os.path.join(BASE_DIR, "data")
MODEL_DIR = os.path.join(BASE_DIR, "model")

DATA_BINARY_DIR = os.path.join(DATA_DIR, "binary_label")
DATA_MULTI_DIR = os.path.join(DATA_DIR, "multi_label")

TRAIN_DATA_DIR = os.path.join(DATA_MULTI_DIR, "translated_train.csv")
TEST_DATA_DIR = os.path.join(DATA_MULTI_DIR, "translated_test.csv")

# Data Check

In [7]:
data_train = pd.read_csv(TRAIN_DATA_DIR, encoding='utf-8')
data_test = pd.read_csv(TEST_DATA_DIR, encoding='utf-8')

X_train = data_train.Text.tolist()
X_test = data_test.Text.tolist()

y_train = data_train.Emotion.tolist()
y_test = data_test.Emotion.tolist()

data = data_train.append(data_test, ignore_index=True)

class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral']

print('size of training set: %s' % (len(data_train['Text'])))
print('size of validation set: %s' % (len(data_test['Text'])))
print(data.Emotion.value_counts())

data.head(10)

size of training set: 7934
size of validation set: 3393
joy        2326
sadness    2317
anger      2259
neutral    2254
fear       2171
Name: Emotion, dtype: int64


,Emotion,Text
0,neutral,내가 더 낫다고 생각하는 다른 그림이 많이 있습니다.
1,sadness,"그러나 개는 늙고 능력이 떨어졌고, 어느 날 길리가 와서 그 개가 뇌졸중을 앓았으므..."
2,fear,티켓을 지불하지 않고 지하철이나 기차에 들어갈 때.
3,fear,이 마지막 부분은 상당한 불안의 근원이 될 수 있으며 처음에는 그러한 명백한 '부도...
4,anger,"그녀는 그가 그들 중 일부에 대해 보여준 친밀함을 싫어했고, 그녀가 일부가 아닌 공..."
5,sadness,우리 가족은 영국에 사는 어머니의 사촌이 임파선 암에 걸렸다 고 우리에게 편지를 썼...
6,joy,내가 중국 실어증에 대한 규범을 수집하도록 선택되었음을 알게 됨 (나는 중국이 신경...
7,anger,"대변인은 ""글렌은 새로운""무정부 상태 ""프로모션이 자신뿐만 아니라 시드 비셔스의 영..."
8,neutral,예 .
9,sadness,화상을 입은 사람을 보면 슬프지만 실제로는 많이 고통을 당해야한다고 생각해서 표현조...


# Training Configuration

In [8]:
##GPU 사용 시
device = torch.device("cuda:0")

In [9]:
# model load
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [10]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [11]:
emotion2label = {"neutral" : 0, "sadness" : 1, "fear" : 2, "anger" : 3, "joy" : 4}
label2emotion = { 0 : "neutral", 1 : "sadness",  2 : "fear",  3 : "anger",  4 : "joy"}

In [12]:
class BERTDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        
        texts = dataset["Text"].tolist()
        labels = dataset["Emotion"].tolist()

        self.sentences = [transform([text]) for text in texts]
        self.labels = [np.int32(emotion2label[label]) for label in labels]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [13]:
## Setting parameters
max_len = 64 ##############################
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 ##############################
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

# Training

In [14]:
data_train = BERTDataset(data_train, tok, max_len, True, False) 
data_test = BERTDataset(data_test, tok, max_len, True, False) 

In [15]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5,
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [21]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [22]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [23]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [24]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [25]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [26]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 1.6236586570739746 train acc 0.234375

epoch 1 train acc 0.47787542273673256


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.7106481481481481


epoch 2 batch id 1 loss 0.9003301858901978 train acc 0.640625

epoch 2 train acc 0.7580238683662851


...


epoch 3 test acc 0.7780671296296297


epoch 4 batch id 1 loss 0.5103631019592285 train acc 0.828125

epoch 4 train acc 0.8981773543184183



epoch 4 test acc 0.7826967592592593


epoch 5 batch id 1 loss 0.34208881855010986 train acc 0.890625

epoch 5 train acc 0.933069393860562



epoch 5 test acc 0.7864583333333334


In [27]:
# 모델 저장
torch.save(model.state_dict(), os.path.join(MODEL_DIR, "multi_label_model.pt"))